In [17]:
import pandas as pd
import numpy as np
import folium
from googletrans import Translator
import mysql.connector as conn

In [2]:

alcohol= pd.read_csv("HappinessAlcoholConsumption.csv")
alcohol

,Country,Region,Hemisphere,HappinessScore,HDI,GDP_PerCapita,Beer_PerCapita,Spirit_PerCapita,Wine_PerCapita
0,Denmark,Western Europe,north,7.526,928,53.579,224,81,278
1,Switzerland,Western Europe,north,7.509,943,79.866,185,100,280
2,Iceland,Western Europe,north,7.501,933,60.530,233,61,78
3,Norway,Western Europe,north,7.498,951,70.890,169,71,129
4,Finland,Western Europe,north,7.413,918,43.433,263,133,97
...,...,...,...,...,...,...,...,...,...
117,Tanzania,Sub-Saharan Africa,south,3.666,533,878.000,36,6,1
118,Liberia,Sub-Saharan Africa,north,3.622,432,455.000,19,152,2
119,Benin,Sub-Saharan Africa,north,3.484,512,789.000,34,4,13
120,Togo,Sub-Saharan Africa,north,3.303,500,577.000,36,2,19


In [11]:
deaths= pd.read_csv("all_weekly_excess_deaths.csv")
deaths

,country,region,region_code,start_date,end_date,days,year,week,population,total_deaths,covid_deaths,expected_deaths,excess_deaths,non_covid_deaths,covid_deaths_per_100k,excess_deaths_per_100k,excess_deaths_pct_change
0,Australia,Australia,0,2019-12-30,2020-01-05,7,2020,1,25788217,2811.2,0,2964.577903,-153.377903,2811.2,0.000000,-0.594760,-0.051737
1,Australia,Australia,0,2020-01-06,2020-01-12,7,2020,2,25788217,2825.8,0,2959.161236,-133.361236,2825.8,0.000000,-0.517140,-0.045067
2,Australia,Australia,0,2020-01-13,2020-01-19,7,2020,3,25788217,2817.9,0,2934.977903,-117.077903,2817.9,0.000000,-0.453998,-0.039891
3,Australia,Australia,0,2020-01-20,2020-01-26,7,2020,4,25788217,2933.3,0,2935.611236,-2.311236,2933.3,0.000000,-0.008962,-0.000787
4,Australia,Australia,0,2020-01-27,2020-02-02,7,2020,5,25788217,2824.7,0,2921.661236,-96.961236,2824.7,0.000000,-0.375990,-0.033187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4732,United States,United States,0,2021-12-06,2021-12-12,7,2021,49,332915074,70157.0,9145,58513.491525,11643.508475,61012.0,2.746947,3.497441,0.198988
4733,United States,United States,0,2021-12-13,2021-12-19,7,2021,50,332915074,69001.0,9222,58909.691525,10091.308475,59779.0,2.770076,3.031196,0.171301
4734,United States,United States,0,2021-12-20,2021-12-26,7,2021,51,332915074,69439.0,10223,59657.691525,9781.308475,59216.0,3.070753,2.938079,0.163957
4735,United States,United States,0,2021-12-27,2022-01-02,7,2021,52,332915074,70532.0,9499,60151.291525,10380.708475,61033.0,2.853280,3.118125,0.172577


In [7]:
#limpieza df medallas, cogemos las columnas que queremos
medallas= medallas[["2","3", "4", "5", "6"]]
medallas

,2,3,4,5,6
0,País,NaN,NaN,NaN,NaN
1,Estados Unidos (USA),1061.0,836.0,738.0,2635.0
2,Unión Soviética (URS),395.0,319.0,296.0,1010.0
3,Reino Unido (GBR),285.0,316.0,315.0,916.0
4,China (CHN),262.0,199.0,173.0,634.0
...,...,...,...,...,...
146,Irak (IRQ),0.0,0.0,1.0,1.0
147,Mauricio (MRI),0.0,0.0,1.0,1.0
148,Togo (TOG),0.0,0.0,1.0,1.0
149,Yibuti (DJI),0.0,0.0,1.0,1.0


In [8]:
#limpieza df medallas, renombramos las columnas que queremos
medallas.rename(columns={'2': 'pais', '3': 'oro', "4": "plata", "5": "bronce", "6":"medallas" }, inplace=True)
medallas

,pais,oro,plata,bronce,medallas
0,País,NaN,NaN,NaN,NaN
1,Estados Unidos (USA),1061.0,836.0,738.0,2635.0
2,Unión Soviética (URS),395.0,319.0,296.0,1010.0
3,Reino Unido (GBR),285.0,316.0,315.0,916.0
4,China (CHN),262.0,199.0,173.0,634.0
...,...,...,...,...,...
146,Irak (IRQ),0.0,0.0,1.0,1.0
147,Mauricio (MRI),0.0,0.0,1.0,1.0
148,Togo (TOG),0.0,0.0,1.0,1.0
149,Yibuti (DJI),0.0,0.0,1.0,1.0


In [30]:
#quitamos la abreviatura de los paises
medallas["pais"]=medallas["pais"].apply(lambda x : x.split("(")[0])
medallas

,pais,oro,plata,bronce,medallas
0,País,NaN,NaN,NaN,NaN
1,Estados Unidos,1061.0,836.0,738.0,2635.0
2,Unión Soviética,395.0,319.0,296.0,1010.0
3,Reino Unido,285.0,316.0,315.0,916.0
4,China,262.0,199.0,173.0,634.0
...,...,...,...,...,...
146,Irak,0.0,0.0,1.0,1.0
147,Mauricio,0.0,0.0,1.0,1.0
148,Togo,0.0,0.0,1.0,1.0
149,Yibuti,0.0,0.0,1.0,1.0


In [33]:
#quitamos los nan
medallas.dropna(inplace=True)

In [34]:
medallas


,pais,oro,plata,bronce,medallas
1,Estados Unidos,1061.0,836.0,738.0,2635.0
2,Unión Soviética,395.0,319.0,296.0,1010.0
3,Reino Unido,285.0,316.0,315.0,916.0
4,China,262.0,199.0,173.0,634.0
5,Alemania,229.0,259.0,282.0,770.0
...,...,...,...,...,...
145,Guyana,0.0,0.0,1.0,1.0
146,Irak,0.0,0.0,1.0,1.0
147,Mauricio,0.0,0.0,1.0,1.0
148,Togo,0.0,0.0,1.0,1.0


In [18]:
#empiezo hacer pruebas de tradcucción para cambiar los nombres de los paises a ingles
translator = Translator()

In [19]:
result = translator.translate('Islandia', src='es', dest='en')
result.text

'Iceland'

In [20]:
#consigo hacer la función para traducir los nombres
def traducir(pais):
    translator = Translator()
    result=translator.translate(pais, src='es', dest='en')
    return result.text

In [23]:
medallas.pais=medallas.pais.apply(traducir)
medallas

,pais,oro,plata,bronce,medallas
0,U.S.,1061.0,836.0,738.0,2635.0
1,Soviet Union,395.0,319.0,296.0,1010.0
2,United Kingdom.,285.0,316.0,315.0,916.0
3,China,262.0,199.0,173.0,634.0
4,Germany,229.0,259.0,282.0,770.0
...,...,...,...,...,...
143,Guyana,0.0,0.0,1.0,1.0
144,Iraq.,0.0,0.0,1.0,1.0
145,Mauricio,0.0,0.0,1.0,1.0
146,Togo,0.0,0.0,1.0,1.0


In [25]:
#quito los puntos después de los nombres de los paises
medallas.pais=medallas.pais.apply(lambda x: x.rstrip('.'))
medallas

,pais,oro,plata,bronce,medallas
0,U.S,1061.0,836.0,738.0,2635.0
1,Soviet Union,395.0,319.0,296.0,1010.0
2,United Kingdom,285.0,316.0,315.0,916.0
3,China,262.0,199.0,173.0,634.0
4,Germany,229.0,259.0,282.0,770.0
...,...,...,...,...,...
143,Guyana,0.0,0.0,1.0,1.0
144,Iraq,0.0,0.0,1.0,1.0
145,Mauricio,0.0,0.0,1.0,1.0
146,Togo,0.0,0.0,1.0,1.0


In [35]:
#he detectado que United States me lo había traducido como U.S, y lo cambio
medallas=medallas.replace(to_replace ="U.S",
                 value ="United States")

In [36]:
#guado csv
medallas.to_csv('medallas.csv', index=False)

In [52]:
#hacemos lo mismo con el dataframe de 'poblacion'
poblacion= pd.read_csv("poblacion.csv")
poblacion

,PAÍS,POBLACIÓN
0,China,"1,439,323,774"
1,India,"1,380,004,385"
2,Estados Unidos,"331,002,647"
3,Indonesia,"273,523,621"
4,Pakistán,"220,892,331"
...,...,...
195,Aruba,"106,766"
196,Tonga,"105,697"
197,Islas Vírgenes de los Estados Unidos,"104,423"
198,Seychelles,"98,340"


In [10]:
poblacion= poblacion[["PAÍS","POBLACIÓN"]]
poblacion

,PAÍS,POBLACIÓN
0,China,"1,439,323,774"
1,India,"1,380,004,385"
2,Estados Unidos,"331,002,647"
3,Indonesia,"273,523,621"
4,Pakistán,"220,892,331"
...,...,...
195,Aruba,"106,766"
196,Tonga,"105,697"
197,Islas Vírgenes de los Estados Unidos,"104,423"
198,Seychelles,"98,340"


In [54]:
poblacion.PAÍS=poblacion.PAÍS.apply(traducir)
poblacion

,PAÍS,POBLACIÓN
0,China,"1,439,323,774"
1,India,"1,380,004,385"
2,U.S,"331,002,647"
3,Indonesia,"273,523,621"
4,Pakistan,"220,892,331"
...,...,...
195,Aruba,"106,766"
196,Tonga.,"105,697"
197,Virgin Islands of the United States,"104,423"
198,Seychelles,"98,340"


In [55]:
poblacion.PAÍS=poblacion.PAÍS.apply(lambda x: x.rstrip('.'))
poblacion

,PAÍS,POBLACIÓN
0,China,"1,439,323,774"
1,India,"1,380,004,385"
2,U.S,"331,002,647"
3,Indonesia,"273,523,621"
4,Pakistan,"220,892,331"
...,...,...
195,Aruba,"106,766"
196,Tonga,"105,697"
197,Virgin Islands of the United States,"104,423"
198,Seychelles,"98,340"


In [56]:
poblacion=poblacion.replace(to_replace ="U.S",
                 value ="United States")

In [14]:
#nos ponemos con el dataframe 'deaths', seleccionamos las columnas
deaths=deaths[["country", "start_date", "end_date", "year", "population", "total_deaths", "covid_deaths", "non_covid_deaths"]]


In [38]:
deaths

,country,start_date,end_date,year,population,total_deaths,covid_deaths,non_covid_deaths
0,Australia,2019-12-30,2020-01-05,2020,25788217,2811.2,0,2811.2
1,Australia,2020-01-06,2020-01-12,2020,25788217,2825.8,0,2825.8
2,Australia,2020-01-13,2020-01-19,2020,25788217,2817.9,0,2817.9
3,Australia,2020-01-20,2020-01-26,2020,25788217,2933.3,0,2933.3
4,Australia,2020-01-27,2020-02-02,2020,25788217,2824.7,0,2824.7
...,...,...,...,...,...,...,...,...
4732,United States,2021-12-06,2021-12-12,2021,332915074,70157.0,9145,61012.0
4733,United States,2021-12-13,2021-12-19,2021,332915074,69001.0,9222,59779.0
4734,United States,2021-12-20,2021-12-26,2021,332915074,69439.0,10223,59216.0
4735,United States,2021-12-27,2022-01-02,2021,332915074,70532.0,9499,61033.0


In [61]:
poblacion.to_csv('poblacion.csv', index=False)

In [62]:
medallas.to_csv('medallas.csv', index=False)

In [63]:
deaths.to_csv('deaths.csv', index=False)

In [ ]:
#deje el el trabajo guardando los csv y cargandolos de nuevo

In [69]:
poblacion= pd.read_csv("poblacion.csv")
alcohol= pd.read_csv("HappinessAlcoholConsumption.csv")
deaths= pd.read_csv("all_weekly_excess_deaths.csv")
medallas= pd.read_csv("medallas.csv")

In [13]:
alcohol


,Country,Region,Hemisphere,HappinessScore,HDI,GDP_PerCapita,Beer_PerCapita,Spirit_PerCapita,Wine_PerCapita
0,Denmark,Western Europe,north,7.526,928,53.579,224,81,278
1,Switzerland,Western Europe,north,7.509,943,79.866,185,100,280
2,Iceland,Western Europe,north,7.501,933,60.530,233,61,78
3,Norway,Western Europe,north,7.498,951,70.890,169,71,129
4,Finland,Western Europe,north,7.413,918,43.433,263,133,97
...,...,...,...,...,...,...,...,...,...
117,Tanzania,Sub-Saharan Africa,south,3.666,533,878.000,36,6,1
118,Liberia,Sub-Saharan Africa,north,3.622,432,455.000,19,152,2
119,Benin,Sub-Saharan Africa,north,3.484,512,789.000,34,4,13
120,Togo,Sub-Saharan Africa,north,3.303,500,577.000,36,2,19


In [64]:
#seleccionamos las columnas de alcohol
alcohol=alcohol[['Country', 'Hemisphere', 'HDI', 'GDP_PerCapita', 'Beer_PerCapita', 'Spirit_PerCapita', 'Wine_PerCapita']]
alcohol

,Country,Hemisphere,HDI,GDP_PerCapita,Beer_PerCapita,Spirit_PerCapita,Wine_PerCapita
0,Denmark,north,928,53.579,224,81,278
1,Switzerland,north,943,79.866,185,100,280
2,Iceland,north,933,60.530,233,61,78
3,Norway,north,951,70.890,169,71,129
4,Finland,north,918,43.433,263,133,97
...,...,...,...,...,...,...,...
117,Tanzania,south,533,878.000,36,6,1
118,Liberia,north,432,455.000,19,152,2
119,Benin,north,512,789.000,34,4,13
120,Togo,north,500,577.000,36,2,19


In [67]:
alcohol.to_csv("alcohol.csv", index=False)

In [60]:
tibus=list(deaths.country.unique())
len(tibus)

46

In [59]:
acohol = alcohol[~alcohol['Country'].isin(tibus)]
alcohol

,Country,Region,Hemisphere,HappinessScore,HDI,GDP_PerCapita,Beer_PerCapita,Spirit_PerCapita,Wine_PerCapita
0,Denmark,Western Europe,north,7.526,928,53.579,224,81,278
1,Switzerland,Western Europe,north,7.509,943,79.866,185,100,280
2,Iceland,Western Europe,north,7.501,933,60.530,233,61,78
3,Norway,Western Europe,north,7.498,951,70.890,169,71,129
4,Finland,Western Europe,north,7.413,918,43.433,263,133,97
...,...,...,...,...,...,...,...,...,...
117,Tanzania,Sub-Saharan Africa,south,3.666,533,878.000,36,6,1
118,Liberia,Sub-Saharan Africa,north,3.622,432,455.000,19,152,2
119,Benin,Sub-Saharan Africa,north,3.484,512,789.000,34,4,13
120,Togo,Sub-Saharan Africa,north,3.303,500,577.000,36,2,19


# Asegurando nombre de países

In [70]:
medallas=medallas.replace(to_replace ="Bosnia and Herzegovina",
                 value ="Bosnia Herzegovina")

In [71]:
poblacion=poblacion.replace(to_replace ="Bahrain",
                 value ="Baréin")

In [72]:
alcohol=alcohol.replace(to_replace ="Bahrain",
                 value ="Baréin")

In [73]:
deaths=deaths.replace(to_replace ="Bahrain",
                 value ="Baréin")

In [74]:
medallas=medallas.replace(to_replace ="Bosnia and Herzegovina", value ="Bosnia Herzegovina")
poblacion=poblacion.replace(to_replace ="Bosnia and Herzegovina", value ="Bosnia Herzegovina")
alcohol=alcohol.replace(to_replace ="Bosnia and Herzegovina", value ="Bosnia Herzegovina")
deaths=deaths.replace(to_replace ="Bosnia and Herzegovina", value ="Bosnia Herzegovina")

In [77]:
medallas=medallas.replace(to_replace ="Britain", value ="United Kingdom")
poblacion=poblacion.replace(to_replace ="Britain", value ="United Kingdom")
alcohol=alcohol.replace(to_replace ="Britain", value ="United Kingdom")
deaths=deaths.replace(to_replace ="Britain", value ="United Kingdom")

In [78]:
medallas=medallas.replace(to_replace ="Chili", value ="Chile")
poblacion=poblacion.replace(to_replace ="Chili", value ="Chile")
alcohol=alcohol.replace(to_replace ="Chili", value ="Chile")
deaths=deaths.replace(to_replace ="Chili", value ="Chile")

In [79]:
medallas=medallas.replace(to_replace ="China Taipei", value ="China")
poblacion=poblacion.replace(to_replace ="China Taipei", value ="China")
alcohol=alcohol.replace(to_replace ="China Taipei", value ="China")
deaths=deaths.replace(to_replace ="China Taipei", value ="China")

In [80]:
medallas=medallas.replace(to_replace ="Comoras", value ="Comoros")
poblacion=poblacion.replace(to_replace ="Comoras", value ="Comoros")
alcohol=alcohol.replace(to_replace ="Comoras", value ="Comoros")
deaths=deaths.replace(to_replace ="Comoras", value ="Comoros")

In [81]:
medallas=medallas.replace(to_replace ="Czech Republic", value ="Czechoslovakia")
poblacion=poblacion.replace(to_replace ="Czech Republic", value ="Czechoslovakia")
alcohol=alcohol.replace(to_replace ="Czech Republic", value ="Czechoslovakia")
deaths=deaths.replace(to_replace ="Czech Republic", value ="Czechoslovakia")

In [82]:
medallas=medallas.replace(to_replace ="Dem. Rep. Congo", value ="Congo")
poblacion=poblacion.replace(to_replace ="Dem. Rep. Congo", value ="Congo")
alcohol=alcohol.replace(to_replace ="Dem. Rep. Congo", value ="Congo")
deaths=deaths.replace(to_replace ="Dem. Rep. Congo", value ="Congo")

In [84]:
medallas=medallas.replace(to_replace ="Cote d'Ivoire", value ="Ivory Coast")
poblacion=poblacion.replace(to_replace ="Cote d'Ivoire", value ="Ivory Coast")
alcohol=alcohol.replace(to_replace ="Cote d'Ivoire", value ="Ivory Coast")
deaths=deaths.replace(to_replace ="Cote d'Ivoire", value ="Ivory Coast")

In [85]:
medallas=medallas.replace(to_replace ="Mauricio", value ="Mauritius")
poblacion=poblacion.replace(to_replace ="Mauricio", value ="Mauritius")
alcohol=alcohol.replace(to_replace ="Mauricio", value ="Mauritius")
deaths=deaths.replace(to_replace ="Mauricio", value ="Mauritius")

In [86]:
medallas=medallas.replace(to_replace ="Netherlands Antilles", value ="Netherlands")
poblacion=poblacion.replace(to_replace ="Netherlands Antilles", value ="Netherlands")
alcohol=alcohol.replace(to_replace ="Netherlands Antilles", value ="Netherlands")
deaths=deaths.replace(to_replace ="Netherlands Antilles", value ="Netherlands")

In [87]:
medallas=medallas.replace(to_replace ="Nigeria", value ="Niger")
poblacion=poblacion.replace(to_replace ="Nigeria", value ="Niger")
alcohol=alcohol.replace(to_replace ="Nigeria", value ="Niger")
deaths=deaths.replace(to_replace ="Nigeria", value ="Niger")

In [88]:
medallas=medallas.replace(to_replace ="Rep. Congo", value ="Congo")
poblacion=poblacion.replace(to_replace ="Rep. Congo", value ="Congo")
alcohol=alcohol.replace(to_replace ="Rep. Congo", value ="Congo")
deaths=deaths.replace(to_replace ="Rep. Congo", value ="Congo")

In [89]:
medallas=medallas.replace(to_replace ="Russian Federation", value ="Russia")
poblacion=poblacion.replace(to_replace ="Russian Federation", value ="Russia")
alcohol=alcohol.replace(to_replace ="Russian Federation", value ="Russia")
deaths=deaths.replace(to_replace ="Russian Federation", value ="Russia")

In [90]:
medallas=medallas.replace(to_replace ="Serbia and Montenegro", value ="Serbia")
poblacion=poblacion.replace(to_replace ="Serbia and Montenegro", value ="Serbia")
alcohol=alcohol.replace(to_replace ="Serbia and Montenegro", value ="Serbia")
deaths=deaths.replace(to_replace ="Serbia and Montenegro", value ="Serbia")

In [91]:
medallas=medallas.replace(to_replace ="Soviet Union", value ="Russia")
poblacion=poblacion.replace(to_replace ="Soviet Union", value ="Russia")
alcohol=alcohol.replace(to_replace ="Soviet Union", value ="Russia")
deaths=deaths.replace(to_replace ="Soviet Union", value ="Russia")

In [92]:
medallas=medallas.replace(to_replace ="SPAIN", value ="Spain")
poblacion=poblacion.replace(to_replace ="SPAIN", value ="Spain")
alcohol=alcohol.replace(to_replace ="SPAIN", value ="Spain")
deaths=deaths.replace(to_replace ="SPAIN", value ="Spain")

In [93]:
medallas=medallas.replace(to_replace ="Suriname", value ="Surinam")
poblacion=poblacion.replace(to_replace ="Suriname", value ="Surinam")
alcohol=alcohol.replace(to_replace ="Suriname", value ="Surinam")
deaths=deaths.replace(to_replace ="Suriname", value ="Surinam")

In [94]:
medallas=medallas.replace(to_replace ="Swiss", value ="Switzerland")
poblacion=poblacion.replace(to_replace ="Swiss", value ="Switzerland")
alcohol=alcohol.replace(to_replace ="Swiss", value ="Switzerland")
deaths=deaths.replace(to_replace ="Swiss", value ="Switzerland")

In [95]:
medallas=medallas.replace(to_replace ="Western Germany", value ="Germany")
poblacion=poblacion.replace(to_replace ="Western Germany", value ="Germany")
alcohol=alcohol.replace(to_replace ="Western Germany", value ="Germany")
deaths=deaths.replace(to_replace ="Western Germany", value ="Germany")

In [96]:
medallas=medallas.replace(to_replace ="East Germany", value ="Germany")
poblacion=poblacion.replace(to_replace ="East Germany", value ="Germany")
alcohol=alcohol.replace(to_replace ="East Germany", value ="Germany")
deaths=deaths.replace(to_replace ="East Germany", value ="Germany")

In [97]:
medallas=medallas.replace(to_replace ="grenade", value ="Grenoble")
poblacion=poblacion.replace(to_replace ="grenade", value ="Grenoble")
alcohol=alcohol.replace(to_replace ="grenade", value ="Grenoble")
deaths=deaths.replace(to_replace ="grenade", value ="Grenoble")

In [100]:
medallas.to_csv('medallas.csv', index=False)
alcohol.to_csv('alcohol.csv', index=False)
deaths.to_csv('deaths.csv', index=False)